# ML bias exercise solutions

This programming exercise is focused on machine learning (ML) bias in applications. The exercise involves designing a ML model to predict whether an individual's income is above or below $50,000 based on demographic and employment-related features such as age, education level, and occupation.

The dataset used for training the model is the Adult Income dataset, which contains information about individuals' demographic and employment-related features, as well as their income level. In this exercise, you will preprocesses the data, train a ML model, and evaluate its performance using metrics such as F1 score and recall.

The exercise also highlights the issue of bias in machine learning, and how it can lead to unequal treatment or discrimination against certain demographic groups. Specifically, the exercise focuses on the bias observed in the model's performance on different demographic groups, such as 'Male' and 'White' compared to 'Female' and 'Black'. The exercise demonstrates how bias can arise in ML models, and emphasizes the importance of addressing bias to ensure fair and equitable treatment of all individuals and groups.

In [65]:
# Import necessary librairies

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

Your task now is to load and read the dataset. The data can be downloaded automatically using this link: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data . The goal is to load it using the **read_csv()** function from the Pandas library. Look at the documentation online for more information about this function. Feel free to explore the data and **note that there is no header**.

In [67]:
# SOLUTION:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
data = pd.read_csv(url, header=None)

Your next task is to assign a name to every feature (column) of your dataset. To do so, you can use the **.column** attribute of the dataframe that you created earlier to assign the name table. The data contains the following features:
* Age
* Work class
* Final weight
* Education
* Years of completed education
* Martiat status
* Occupation
* Family relationship
* Race
* Sex
* Capital gain
* Capital loss
* Hours of work per week
* Native country
* Net income

You can look at the data to understand them (like the type) or look online. Please choose a relevant name for every feature according to this description.

In [69]:
# SOLUTION:
data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
                'marital_status', 'occupation', 'relationship', 'race', 'sex',
                'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

You probably seen that the data contains a lot of dummy variables. These variables (called dummies sometimes) contain values that can easily be translated to binary or one-hot vectors (look online if needed). The income can be translated into a binary variable. Work class, education, martial status, occupation, family relationship, race, sex and native country can be raplaced by one-hot vectors. Complete the code below by replacing *INCOME_FEATURE_NAME* and *LIST_OF_DUMMY_FEATURES_NAME* by their correponding names that you choosed above.

In [70]:
# SOLUTION:
le = LabelEncoder()
data['income'] = le.fit_transform(data['income'])

data = pd.get_dummies(data, columns=['workclass', 'education', 'marital_status',
                                     'occupation', 'relationship', 'race', 'sex',
                                     'native_country'])

Now, we want to train our first ML model. To do so, we first need to split our data to a training and testing set. The training set must represent 80% of the total data and 20% for the testing set. This can be done easily by using the **train_test_split()** imported from the *sklearn* library [more info here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). The training and testing set are composed of the labels (the income in this case) that we want to predict, and the features (all data except income). The income feature can be easily removed from the feature by using the **drop()** function from *Pandas*. Please complete the call below with the right parameters.

In [71]:
# SOLUTION:
train_data, test_data, train_labels, test_labels = train_test_split(data.drop('income', axis=1), data['income'], test_size=0.2, random_state=42)

Now that the data is correctly splitted, it is finally time to train our ML model. We will use a logistic regression function that is used often for this kind of classifiction. Create a model by using the *LogisticRegression()* function from *sklearn* library [more info here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and store it in a variable. In this exercise, the only parameter that you need to pass is the maximum number of iteration for convergence *max_iter*. Please set it to 1000 to ensure convergence.

Then, fit your model by applying the **fit()** function to the model you just created by passing the training features *train_data* and training labels *train_labels* you previously created.

In [72]:
# SOLUTION:
# Train a logistic regression model on the training data
model = LogisticRegression(max_iter=1000)
model.fit(train_data, train_labels)

LogisticRegression(max_iter=1000)

Now that our model is trained, it is time to test it. Use the **predict()** function from your model by passing the testing set *test_data* as a parameter and store the result in the *test_preds* variable. Then print the classification report.

In [73]:
# SOLUTION:
test_preds = model.predict(test_data)
print('Test Set Classification Report:\n', classification_report(test_labels, test_preds))

Test Set Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.97      0.88      4942
           1       0.73      0.27      0.39      1571

    accuracy                           0.80      6513
   macro avg       0.77      0.62      0.64      6513
weighted avg       0.79      0.80      0.76      6513



What can you conclude with this model, knowing that:
* 0 means income <= 50K
* 1 means income > 50K
* precision means accuracy of the prediction
* recall is the fraction of relevant instances that are retrieved by the model.
* F1 score is the harmonic mean of precision and recall.
* support is the number of corresponding sample in each category

#### Solution: We see a higher precision for lower income (0.81 against 0.73 for higher income). This model is more likely to predict a lower income probabely because of its higher support. Thus, this model is biased.

Now, we can go further in our analysis by comparing the classification scores for black and white people separately. 
Try to understand, replace RACE_FEATURE_NAME by the name you choosed for this feature and execute the code. Can you see any bias ? If yes, what are your suggestion to improve this model?

In [75]:
# SOLUTION:
race_cols = ['race_ Black', 'race_ White']
for race in race_cols:
    race_test_mask = test_data[race] == 1
    race_test_data = test_data[race_test_mask]
    race_test_labels = test_labels[race_test_mask]
    race_test_preds = model.predict(race_test_data)
    print(f'Classification Report for {race} individuals:\n', 
          classification_report(race_test_labels, race_test_preds))

Classification Report for race_ Black individuals:
               precision    recall  f1-score   support

           0       0.92      0.99      0.95       534
           1       0.71      0.26      0.38        65

    accuracy                           0.91       599
   macro avg       0.81      0.62      0.67       599
weighted avg       0.89      0.91      0.89       599

Classification Report for race_ White individuals:
               precision    recall  f1-score   support

           0       0.79      0.97      0.87      4167
           1       0.73      0.27      0.39      1428

    accuracy                           0.79      5595
   macro avg       0.76      0.62      0.63      5595
weighted avg       0.78      0.79      0.75      5595



#### Solution: As we can see, the category with the higher precision is the black people with lower income (0.92). It means the model is more likely to predict correctly a lower income in this category. It is the same for white people, but the support is much higher, thus the difference of accuracy is clearly lower (0.06). An idea could be to collect more data for less represented category in a way to have equal proportions, or by reducing some categories to by balanced (but a lot of data is needed).

If you want to go further in this analysis, feel free to isolate other categories like the sex (*sex_ Male* and *sex_ Female*) to see eventual other biases.